In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
mnist_args = dotdict()
mnist_args.train = dotdict()
mnist_args.predict = dotdict()

mnist_args.name = 'mnist'
mnist_args.train.batch_size = 100
mnist_args.predict.batch_size = 1000

In [15]:
args = dotdict()
args.data = mnist_args

args.train_epochs= 3
args.lr = 0.001
args.criterion = 'CE'
args.use_gpu = True

In [16]:
driver = ABC_Driver(args)

Use GPU: cuda:None


In [17]:
driver.train()

epoch: 0, train_loss: 1.544973390897115
epoch: 1, train_loss: 1.4905752154191334
epoch: 2, train_loss: 1.4832054245471955


In [18]:
driver.metric()

0.9805

In [ ]:
predict = torch.utils.data.DataLoader(torchvision.datasets.MNIST('../../data/ABC/mnist', train=False, download=True,
                                                  transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))])),
                                                  batch_size=1000, shuffle=True)

In [16]:
a.argmax(axis=0)

tensor([0, 0])

In [17]:
y_true = [1,2,4]

In [18]:
y_pred = [1,0,1]

In [23]:
metric

Accuracy    0.333333
F1               NaN
Recall           NaN
dtype: float64